In [2]:
from datetime import date

from src.domain.services.futures_data_service import FuturesDataService
from src.infrastructure.database.database import db_session
from src.infrastructure.mysql.futures_data_repository_mysql import FuturesDataRepositoryMysql


futures_data_repository = FuturesDataRepositoryMysql(db_session())
futures_data_service = FuturesDataService(futures_data_repository)

asset_name = 'crude_oil'
trade_dates = [date(2024, 4, 2), date(2024, 4, 3), date(2024, 4, 4)]

df = futures_data_service.make_dataframe(asset_name, trade_dates)
df

2024-04-08 07:02:43,385 INFO sqlalchemy.engine.Engine 
            SELECT
                a.id AS asset_id,
                a.name AS asset_name,
                s.trade_date,
                s.month,
                s.settle,
                v.total_volume AS volume,
                v.at_close AS open_interest
            FROM assets a
            JOIN settlements s ON a.id = s.asset_id
            JOIN volume_oi v ON a.id = v.asset_id AND s.trade_date = v.trade_date AND s.month = v.month
            WHERE a.name = %(asset_name)s AND s.trade_date = %(trade_date)s
            
[INFO] 2024-04-08 07:02:43,385 - 
            SELECT
                a.id AS asset_id,
                a.name AS asset_name,
                s.trade_date,
                s.month,
                s.settle,
                v.total_volume AS volume,
                v.at_close AS open_interest
            FROM assets a
            JOIN settlements s ON a.id = s.asset_id
            JOIN volume_oi v ON a.id = v.asset

,trade_date,month,settle,volume,open_interest
0,2024-04-02,2024-05-01,85.15,344040,326287
1,2024-04-02,2024-06-01,84.22,197832,250581
2,2024-04-02,2024-07-01,83.29,108351,163051
3,2024-04-02,2024-08-01,82.34,62665,75100
4,2024-04-02,2024-09-01,81.38,66126,106558
...,...,...,...,...,...
166,2024-04-04,2030-06-01,63.81,0,51
167,2024-04-04,2030-12-01,63.66,1,885
168,2024-04-04,2031-12-01,63.49,1,64
169,2024-04-04,2032-12-01,63.41,0,1010


In [3]:
df = df.sort_values(by=['trade_date', 'month'])

df['settle_spread'] = df.groupby('trade_date')['settle'].diff().fillna(0)
df

,trade_date,month,settle,volume,open_interest,settle_spread
0,2024-04-02,2024-05-01,85.15,344040,326287,0.00
1,2024-04-02,2024-06-01,84.22,197832,250581,-0.93
2,2024-04-02,2024-07-01,83.29,108351,163051,-0.93
3,2024-04-02,2024-08-01,82.34,62665,75100,-0.95
4,2024-04-02,2024-09-01,81.38,66126,106558,-0.96
...,...,...,...,...,...,...
166,2024-04-04,2030-06-01,63.81,0,51,-0.15
167,2024-04-04,2030-12-01,63.66,1,885,-0.15
168,2024-04-04,2031-12-01,63.49,1,64,-0.17
169,2024-04-04,2032-12-01,63.41,0,1010,-0.08
